# INFO 4604 Final Project - Predicting if cancer is benign or not

## Amogh Jahagirdar and Ryan Rouleau

Dataset: [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/home](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/home)

## Precursor Analysis/General Data Cleansing

In [1]:
import pandas as pd

data = pd.read_csv('./data/data.csv')

In [2]:
#Basic summary statistics
data.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,0.0
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,NaN
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,NaN
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,NaN
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,NaN
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,NaN


In [3]:
#Drop last column ("Unnamed:32" column of NaNs being read, when the CSV is opened up in Excel that column doesn't exist)
data_cleaned = data.iloc[:, :-1]
#Drop ID (just a bookeeping column part of the original data)
data_cleaned = data_cleaned.drop("id", 1)

In [4]:
#Analyze the balance of the data.
nrows = data_cleaned.shape[0]
print("Percentages of benign and maligant data is \n {}".format(100 * data_cleaned["diagnosis"].value_counts()/nrows))

Percentages of benign and maligant data is 
 B    62.741652
M    37.258348
Name: diagnosis, dtype: float64


As one can see, there are significantly more benign cases than malignant in the given dataset.

In [5]:
from sklearn.model_selection import train_test_split

X = data_cleaned[data_cleaned.columns.difference(["diagnosis"])]
y = data_cleaned['diagnosis']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25,random_state=123)

## Baseline Model

Next, we will create a simple baseline classifier with no feature extraction. We can use scikit learn's DummyClassifier class with "the most frequent" strategy. This is not used for actual classification purposes, it is mereley a benchmark for what a theoretical classifier would predict if it didn't actually learn from the features in the data (a minimum accuracy for our actual models). All of our models should perform much better than the DummyClasifier.

In [6]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

baseline = DummyClassifier(strategy='most_frequent', random_state=1234)
baseline.fit(X_train, Y_train)

print("Baseline Training accuracy: %0.6f" % accuracy_score(Y_train, baseline.predict(X_train)))
print("Baseline Testing accuracy: %0.6f" % accuracy_score(Y_test, baseline.predict(X_test)))

Baseline Training accuracy: 0.629108
Baseline Testing accuracy: 0.622378


Our models should be able to perform significantly above 60% accuracy.

## Classification Algorithms (Baseline)
###  Decision Tree Classifier

In [7]:
from sklearn.tree import DecisionTreeClassifier

decisionTree = DecisionTreeClassifier(random_state=1234)
decisionTree.fit(X_train, Y_train)

print("Baseline Decision Tree Training accuracy: %0.6f" % accuracy_score(Y_train, decisionTree.predict(X_train)))
print("Baseline Decision Tree Testing accuracy: %0.6f" % accuracy_score(Y_test, decisionTree.predict(X_test)))

Baseline Decision Tree Training accuracy: 1.000000
Baseline Decision Tree Testing accuracy: 0.958042


As seen above, a decision tree classifier with no modification of hyperparamters severly overfits with a training accuracy of exactly 100%.

... hyperparameter selection w/ cv here

### Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression(random_state=1234)
logisticRegression.fit(X_train, Y_train)

print("Baseline Logistic Regression Training accuracy: %0.6f" % accuracy_score(Y_train, logisticRegression.predict(X_train)))
print("Baseline Logistic Regression Testing accuracy: %0.6f" % accuracy_score(Y_test, logisticRegression.predict(X_test)))

Baseline Logistic Regression Training accuracy: 0.948357
Baseline Logistic Regression Testing accuracy: 0.986014


Logistic Regression has learned the `most frequent` strategy that is also used in our baseline without hyperparameter modifications.

... hyperparameter selection w/ cv here

### Support Vector Machine

In [9]:
from sklearn.svm import SVC 

svm = SVC(random_state=1234)
svm.fit(X_train, Y_train)

print("Baseline Support Vector Machine Training accuracy: %0.6f" % accuracy_score(Y_train, svm.predict(X_train)))
print("Baseline Support Vector Machine Testing accuracy: %0.6f" % accuracy_score(Y_test, svm.predict(X_test)))

Baseline Support Vector Machine Training accuracy: 1.000000
Baseline Support Vector Machine Testing accuracy: 0.622378


Using a support vector machine without any hyperparameter modifications also severly overfits with a training accuracy of 100%.  The test accuracy is concerning as it is exactly the same as `most frequent` baseline classifier.

### Neural Net

In [10]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=1234,max_iter=1000)
mlp.fit(X_train, Y_train)

print("Baseline Neural Net Training accuracy: %0.6f" % accuracy_score(Y_train, mlp.predict(X_train)))
print("Baseline Neural Net Testing accuracy: %0.6f" % accuracy_score(Y_test, mlp.predict(X_test)))

Baseline Neural Net Training accuracy: 0.537559
Baseline Neural Net Testing accuracy: 0.531469


# Feature preprocessing via Standard Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train_std = std_scaler.fit_transform(X_train)
X_test_std = std_scaler.transform(X_test)

decisionTree.fit(X_train_std, Y_train)

#A little sanity check to see how models perform after scaling
models = [logisticRegression, decisionTree, mlp, svm]
for model in models:
    #Warm start by default is off so by calling fit it "retrains from scratch" which is what we want
    model.fit(X_train_std, Y_train)
    model_name = model.__class__.__name__
    train_accuracy = accuracy_score(Y_train, model.predict(X_train_std))
    test_accuracy = accuracy_score(Y_test, model.predict(X_test_std))
    print("Train accuracy for model {} after standardizing features: {}".format(model_name, train_accuracy))
    print("Test accuracy for model {} after standardizing features: {}".format(model_name, test_accuracy))

Train accuracy for model LogisticRegression after standardizing features: 0.9859154929577465
Test accuracy for model LogisticRegression after standardizing features: 0.993006993006993
Train accuracy for model DecisionTreeClassifier after standardizing features: 1.0
Test accuracy for model DecisionTreeClassifier after standardizing features: 0.958041958041958
Train accuracy for model MLPClassifier after standardizing features: 0.9929577464788732
Test accuracy for model MLPClassifier after standardizing features: 0.986013986013986
Train accuracy for model SVC after standardizing features: 0.9835680751173709
Test accuracy for model SVC after standardizing features: 0.986013986013986


## Feature Extraction and CV 

#### Perform feature selection using chi^2.
#### Maintain a map between model type and a list of potential params e.e {"logistic_regression": [list of potential c values], "neural net": [different hidden_neural_net sizes], etc}


In [12]:
'''
WARNING: THIS MAY TAKE A WHILE TO RUN
'''

from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.feature_selection import SelectPercentile, f_classif



model_to_possible_params = {}

#Populate model_to_params:
#Key is model_name, value is a dictionary mapping from parameter to a list of potential values
for model in models:
    model_name = model.__class__.__name__
    tunable_params = None
    if model_name == 'LogisticRegression':
        tunable_params = [{'C': [0.001, 0.01,0.1,1,10,100]}]
    elif model_name == 'DecisionTreeClassifier':
        tunable_params = [{'max_depth':[1,2,4,8]}, {'min_samples_leaf': [1,2,3,5,8]}]
    elif model_name == 'SVC':
        tunable_params = [{'kernel': ['rbf', 'linear', 'poly', 'sigmoid']}, {'C': [0.001, 0.01, 0.1, 1, 10, 100]}]
    else:
        tunable_params = [{'activation':['identity', 'logistic', 'tanh', 'relu']}, 
                          {'alpha': [1e-04, 1e-03, 1e-02, 0.05, 1, 10]}, 
                          {'hidden_layer_sizes': [(3), (3,5), (3,5,3), (3,5,5)]}]
    
    model_to_possible_params[model_name] = tunable_params
    
percentiles = [1, 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

#mapping from tuple (percentil, model) to cross validation score

accuracy_results = {}

for model in models:
    
    relevant_params = model_to_possible_params[model.__class__.__name__]
    
    for percentile in percentiles:
    #ToDo: Iterate over percentile selections
        selection = SelectPercentile(percentile=percentile, score_func=f_classif)
        X_train_selected = selection.fit_transform(X_train_std, Y_train)
        gs_classifier = GridSearchCV(model, relevant_params, cv=5, n_jobs=4)
        gs_classifier.fit(X_train_selected, Y_train)
        accuracy_results[(percentile, gs_classifier)] = gs_classifier.best_score_
    


/home/amogh/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/amogh/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/amogh/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/amogh/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged 

In [23]:
accuracy_results

{(1, GridSearchCV(cv=5, error_score='raise',
         estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=1234, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False),
         fit_params=None, iid=True, n_jobs=4,
         param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100]}],
         pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
         scoring=None, verbose=0)): 0.9084507042253521,
 (2, GridSearchCV(cv=5, error_score='raise',
         estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=1234, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False),
         fit_params=None, iid=True, n_jobs=4,
         param_grid=[{'C': [0.001, 0.01, 0.1, 

It's kind of interesting to note, in general more features results in a better accuracy. The best models usually used 100 percentile of features (all features). This implies that there is little correlation between the features (little "redundancy" in feature set). We can do a quick check of the correlation matrix to see if any features are strongly correlated.

In [36]:
%matplotlib inline  

import matplotlib.pyplot as plt

corr = data_cleaned.corr()
print(corr)

                         radius_mean  texture_mean  perimeter_mean  area_mean  \
radius_mean                 1.000000      0.323782        0.997855   0.987357   
texture_mean                0.323782      1.000000        0.329533   0.321086   
perimeter_mean              0.997855      0.329533        1.000000   0.986507   
area_mean                   0.987357      0.321086        0.986507   1.000000   
smoothness_mean             0.170581     -0.023389        0.207278   0.177028   
compactness_mean            0.506124      0.236702        0.556936   0.498502   
concavity_mean              0.676764      0.302418        0.716136   0.685983   
concave points_mean         0.822529      0.293464        0.850977   0.823269   
symmetry_mean               0.147741      0.071401        0.183027   0.151293   
fractal_dimension_mean     -0.311631     -0.076437       -0.261477  -0.283110   
radius_se                   0.679090      0.275869        0.691765   0.732562   
texture_se                 -

# 

In [37]:
sorted_by_accuracy = sorted(accuracy_results.items(), key=lambda kv: kv[1])
#best worst models are tuple ((percentile, model), accuracy)
best_model_tup = sorted_by_accuracy[-1]
worst_model_tup = sorted_by_accuracy[0]
print("The best model according to cross validation results was {} with accuracy {}, percentile features {}".format(best_model_tup[0][1].estimator, best_model_tup[1], best_model_tup[0][0]))
print("The worst model according to cross validation results was {} with accuracy {}, percentile features {}".format(worst_model_tup[0][1].estimator, worst_model_tup[1], worst_model_tup[0][0]))

The best model according to cross validation results was LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1234, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) with accuracy 0.9812206572769953, percentile features 100
The worst model according to cross validation results was DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1234,
            splitter='best') with accuracy 0.903755868544601, percentile features 1


In [40]:
best_model = best_model_tup[0][1].estimator
best_model.fit(X_train_std, Y_train)
accuracy_score(Y_test, best_model.predict(X_test_std))

0.993006993006993

# Error analysis 